In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy
import geopandas as gpd
import xarray as xr
import sparse
from itertools import product, combinations
from pathlib import Path
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib
from collections import defaultdict

pd.options.display.max_rows = 500
#pd.options.display.max_columns = 4000

import sys
sys.path.append('../src/')

from extended_survey import process_people_df, process_places_df, categorize_p, categorize_v
from census import process_census
from constraints import get_ind_const, get_viv_const

survey_dir = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/')
personas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Personas19.CSV')
viviendas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Viviendas19.CSV')
census_iter_path = Path('../data/census_loc/ITER_19CSV20.csv')
census_resageburb_path = Path('../data/census_ageb_manz/RESAGEBURB_19CSV20.csv')
output_path = Path('../output/')

In [2]:
personas = process_people_df(personas_path)
viviendas = process_places_df(viviendas_path)

pcat = personas[[
    'ID_PERSONA', 'ID_VIV', 'FACTOR', 'MUN',
    'SEXO', 'EDAD',
    # 'ENT_PAIS_NAC',
    # 'AFRODES',
    'DHSERSAL1', 'DHSERSAL2', 'RELIGION',
    # 'DIS_VER', 'DIS_OIR', 'DIS_CAMINAR', 'DIS',
    # 'DIS_RECORDAR', 'DIS_BANARSE', 'DIS_HABLAR', 'DIS_MENTAL',
    # 'HLENGUA',
    # 'HESPANOL',  # Global seed zero problem 
    'ASISTEN', 'NIVACAD', 'ESCOLARI', 'ALFABET',
    # 'ENT_PAIS_RES_5A',
    'SITUA_CONYUGAL', 'CONACT',
    'INGTRMEN', 'HORTRA'
]].copy()
vcat = viviendas.copy()

# Look for viviendas with NA
na_vivs_v = vcat.ID_VIV[vcat.isna().T.sum() > 0].to_list()
na_vivs_p = pcat.ID_VIV[pcat.isna().T.sum() > 0].to_list()
na_vivs = set(na_vivs_v + na_vivs_p)

# Drop NA before categorizing
pcat = pcat[~pcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)
vcat = vcat[~vcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)

pcat = categorize_p(pcat)
vcat = categorize_v(vcat)

# Dopr NA again after discretizing
na_vivs = pcat.ID_VIV[pcat.isna().T.sum() > 0].to_list()
pcat = pcat[~pcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)
vcat = vcat[~vcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)

assert pcat.isna().sum().sum() == 0
assert vcat.isna().sum().sum() == 0

# Leave only constrained columns
pcat = pcat[[
    'ID_PERSONA', 'ID_VIV', 'FACTOR',
    'MUN',
    'SEXO', 'EDAD',
    # 'ENT_PAIS_NAC',
    # 'AFRODES',
    #'RELIGION',
    # 'DIS_VER', 'DIS_OIR', 'DIS_CAMINAR',
    # 'DIS_RECORDAR', 'DIS_BANARSE', 'DIS_HABLAR', 'DIS_MENTAL',
    # 'DIS_CON', 'DIS_LIMI',
    # 'HLENGUA',
    # 'HESPANOL',
    'ASISTEN',  'EDUC',
    # 'ALFABET',
    # 'ENT_PAIS_RES_5A',
    'SITUA_CONYUGAL',
    'CONACT',
    # 'DHSERSAL_IMSS', 'DHSERSAL_ISSSTE', 'DHSERSAL_ISSSTE_E', 'DHSERSAL_P_D_M',
    # 'DHSERSAL_Popular_NGenración_SBienestar',
    # 'DHSERSAL_IMSS_Prospera/Bienestar',
    'DHSERSAL_Privado', 'DHSERSAL_Otro',
    'DHSERSAL_No afiliado', 'DHSERSAL_PUB', 'DHSERSAL_AFIL',
]]

vcat = vcat[[
    'ID_VIV', 'FACTOR', 'MUN', 'NUMPERS',
    'CLAVIVP',
    # 'PISOS',
    'CUADORM', 'TOTCUART',
    # 'ELECTRICIDAD', 'AGUA_ENTUBADA',
    # 'ABA_AGUA_ENTU',
    # 'TINACO', 'CISTERNA',
    # 'SERSAN',
    # 'CONAGUA',
    # 'DRENAJE',
    'REFRIGERADOR', 'LAVADORA', 'HORNO',
    'AUTOPROP', 'MOTOCICLETA', 'BICICLETA', 'RADIO', 'TELEVISOR',
    'COMPUTADORA', 'TELEFONO', 'CELULAR', 'INTERNET', 'SERV_TV_PAGA',
    'SERV_PEL_PAGA', 'CON_VJUEGOS',
    'JEFE_SEXO'
]]

print(pcat.shape[0]/personas.shape[0])
print(vcat.shape[0]/viviendas.shape[0])

0.95276327279856
0.9579107969282409


In [3]:
constraints_ind = get_ind_const()
constraints_viv = get_viv_const()

In [4]:
(
    df_mun, df_loc,
    df_agebs, df_agebs_min, df_agebs_max
) = process_census(census_iter_path, census_resageburb_path)

In [7]:
# Build matrices

In [5]:
from setup_lin_system import make_init_system, get_W

In [6]:
X, I, J, L, W, Up, Uh, U, C, Y = make_init_system(pcat, vcat, constraints_ind, constraints_viv, df_mun)

In [7]:
X.shape[0]/pcat.shape[0]

0.10308545561676379

In [8]:
C.shape

(51, 63)

In [9]:
mun_list = X.MUN.unique()
const_zeroprob_list = []
for mun in mun_list:
    mun_mask = Y.MUN == mun
    U_mun = U.loc[:, mun_mask]
    const_zeroprob_list.extend(U_mun.index[U_mun.T.sum() == 0].to_list())
set(const_zeroprob_list)

set()

In [10]:
from ortools.sat.python import cp_model

In [129]:
# Import TAZ
mun = 'Monterrey'

taz_gdf = gpd.read_file('taz_census.gpkg', layer=mun)
print(f'{mun} has {taz_gdf.shape[0]} taz')

# Total number of households
N_mun = int(df_mun.loc[mun, 'TVIVHAB'])

# Load the constraints as a dictionary
C_mun = C.loc[mun].astype(int).to_dict()
C_taz_all = taz_gdf.set_index('ZONA')[C_mun.keys()].fillna(0).astype(int)

# Get the sample households ids and the contraint weight matrix
Y_mun = Y.index[Y[mun] > 0]
U_mun = U.loc[:, Y_mun]

# Reduce the number of variables by identifying equivalent households
# These are equivalent from the perspective of the solver, since they
# contribute equally to the constraints.
# Other attributes may differ, but can be recovered in post-processing and
# assigned accordingly to the sample proportions.
gcols = U_mun.index.to_list()
H = U_mun.T.reset_index().groupby(gcols).agg({'ID_VIV': list}).reset_index()
Uh = H.drop(columns='ID_VIV').T
h_to_y = H.ID_VIV
Yh = h_to_y.index
print(f'{len(Y_mun)} orignal households compress into {len(Yh)} distinct prototypes.')

Monterrey has 89 taz
4794 orignal households compress into 3296 distinct prototypes.


In [126]:
def create_vars(model, Y, min_val, max_val, prefix=''):
    y = {}
    for var_id in Y:
        y[var_id] = model.NewIntVar(min_val, max_val, f'{prefix}{var_id}')
    return y

In [127]:
def create_const(model, C_dict, U, y, relax_factor=1):
    for c_name, c_val in C_dict.items():
        coefficients = U.loc[c_name][U.loc[c_name] > 0]
        expressions = [y[vid] for vid in coefficients.index]
        if c_name in ['POBTOT', 'TVIVHAB']:
            model.Add(cp_model.LinearExpr.WeightedSum(expressions, coefficients.values) == c_val)
        else:
            model.Add(cp_model.LinearExpr.WeightedSum(expressions, coefficients.values) >= relax_factor*c_val)

In [154]:
model = cp_model.CpModel()

# Create municipality level variables
y_mun = create_vars(model, Yh, 1, N_mun)

# Add municipality level constraints
create_const(model, C_mun, Uh, y_mun)


In [155]:
# Load TAZ into model
y_taz = {}
for taz, C_taz in C_taz_all.iterrows():
    C_taz = C_taz.to_dict()
    N_taz = C_taz['TVIVHAB']
    if N_taz == 0:
        print(f'Ignoring empty taz {taz}.')
        continue
    assert N_taz > 0
    
    # Add variables and constraints for specific taz
    y_taz[taz] = create_vars(model, Yh, 0, N_taz, f'{taz}_')
    create_const(model, C_taz, Uh, y_taz[taz], relax_factor=1)

Ignoring empty taz 8.
Ignoring empty taz 37.
Ignoring empty taz 50.


In [ ]:
# Create taz to mun hierarchichal constraints
for var_id, var_mun in y_mun.items():
    var_tazs = []
    for taz_dict in y_taz.values():
        var_tazs.append(taz_dict[var_id])
    model.Add(sum(var_tazs) == var_mun)

In [156]:
solver = cp_model.CpSolver()

#solver.parameters.log_search_progress = True
#solver.log_callback = print
#solver.parameters.linearization_level = 0

status = solver.Solve(model)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution found.')
else:
    print("No solution found.")

No solution found.


In [182]:
# Solving each taz independently
model = cp_model.CpModel()

taz = 4
C_taz = C_taz_all.loc[taz]
N_taz = C_taz['TVIVHAB']
assert N_taz > 0
    
# Add variables and constraints for specific taz
y_taz = create_vars(model, Yh, 0, N_taz)
create_const(model, C_taz, Uh, y_taz, relax_factor=1)

print(model.ModelStats())

solver = cp_model.CpSolver()

#solver.parameters.log_search_progress = True
#solver.log_callback = print
#solver.parameters.linearization_level = 0

status = solver.Solve(model)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution found.')
else:
    print("No solution found.")

satisfaction model '': (model_fingerprint: 0x2b4cb970cc635376)
#Variables: 3'296
  - 3'296 in [0,393]
#kLinearN: 63 (#terms: 99'996)
Solution found.


In [180]:
taz_gdf[['ZONA', 'POBTOT', 'TVIVHAB']].sort_values('TVIVHAB')

,ZONA,POBTOT,TVIVHAB
54,54,156.0,42.0
10,10,301.0,113.0
0,-10,953.0,285.0
31,31,898.0,332.0
4,4,1058.0,393.0
55,55,2185.0,527.0
21,21,2838.0,550.0
66,66,2471.0,583.0
62,62,2157.0,676.0
56,56,2898.0,751.0
